<h1 id="Contents">Contents<a href="#Contents"></a></h1>
        <ol>
        <li><a class="" href="#Imports">Imports</a></li>
<li><a class="" href="#Introduction">Introduction</a></li>
<ol><li><a class="" href="#Assumptions">Assumptions</a></li>
<ol><li><a class="" href="#Assets-Assumptions">Assets Assumptions</a></li>
<li><a class="" href="#Market-Assumptions">Market Assumptions</a></li>
</ol></ol><li><a class="" href="#Black-Scholes-Equation">Black-Scholes Equation</a></li>
<ol><li><a class="" href="#Notations">Notations</a></li>
<ol><li><a class="" href="#General-and-Market-Parameters">General and Market Parameters</a></li>
<li><a class="" href="#Asset-Parameters">Asset Parameters</a></li>
<li><a class="" href="#Option-Parameters">Option Parameters</a></li>
</ol><li><a class="" href="#Differential-Equation">Differential Equation</a></li>
<li><a class="" href="#Solution-and-the-Black-Scholes-Formula">Solution and the Black-Scholes Formula</a></li>
<ol><li><a class="" href="#Call-Option">Call Option</a></li>
<li><a class="" href="#Put-Option">Put Option</a></li>
<li><a class="" href="#An-Example-Calculation">An Example Calculation</a></li>
</ol><li><a class="" href="#Python-Implementation">Python Implementation</a></li>
</ol><li><a class="" href="#Greeks">Greeks</a></li>
<ol><li><a class="" href="#Mathematical-Form-of-Greeks">Mathematical Form of Greeks</a></li>
<ol><li><a class="" href="#Greeks-for-Call">Greeks for Call</a></li>
<li><a class="" href="#Greeks-for-Put">Greeks for Put</a></li>
</ol></ol><li><a class="" href="#Volatility">Volatility</a></li>
<ol><li><a class="" href="#Log-Returns">Log Returns</a></li>
<ol><li><a class="" href="#Adding-Log-Returns">Adding Log Returns</a></li>
</ol><li><a class="" href="#Calculating-Volatility">Calculating Volatility</a></li>
<ol><li><a class="" href="#Using-Simple-Returns">Using Simple Returns</a></li>
<li><a class="" href="#Using-the-Log-Return">Using the Log Return</a></li>
<li><a class="" href="#Using-the-Black-Scholes-Model">Using the Black-Scholes Model</a></li>
</ol>

# Imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from scipy.stats import norm
import datetime

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
pio.templates.default = "plotly_dark"

# Introduction

Developed in 1973 by Fischer Black, Robert Merton, and Myron Scholes, the Black-Scholes model was the first widely used mathematical method to calculate the theoretical value of an option contract, using current stock prices, expected dividends, the option's strike price, expected interest rates, time to expiration, and expected volatility.

The model shows that the option has a unique price given the risk of the security and its expected return. The model is based on a partial differential equation (PDE), the so-called Black-Scholes equation, from which one can deduce the Black-Scholes formula, which gives a theoretical estimate of the correct price of European stock options.

## Assumptions

The original Black-Scholes model is based on a core assumption that the market consists of at least one risky asset (such as a stock) and one (essentially) risk-free asset, such as a money market fund, cash or a government bond. In addition, it assumes three properties of the two assets, and four of the market itself.

### Assets Assumptions

Assumptions about the assets in the market are: 

1. The rate of return on the risk-free asset is constant and known. This behaves as interest rate.

1. The riky asset follows geometric Brownian motion, which is a continuous-time stochastic process. That is, the instantaneous log return of the risky asset’s price is assumed to behave as an infinitesimal random walk with constant drift and volatility

3. The risky asset pays no dividends.

### Market Assumptions

Here are the four market assumptions:

1. The market is complete, which means that there are no arbitrage (risk-free profit) opportunities.
2. It is possible to borrow and lend any amount of cash at the same rate as the interest rate of the risk-free asset
3. It is possible to buy and sell any amount of the stock (including short selling)
4. There are no transaction costs in the market (i.e. no commission for buying or selling securities or derivative instruments)

Of course, the assumptions are not realistic, but they are useful for the purpose of the model.

In subsequent extensions of the original model, these assumptions have been revised to adjust for dynamic interest rates for the risk-free asset (Merton, 1976), transaction costs for buying and selling (Ingersoll, 1976) and dividend payouts for the risky asset (Whaley, 1981).

# Black-Scholes Equation

With these assumptions, suppose there is a derivative security also trading in this market. It is specified that this security will have a certain payoff at a specified date in the future, depending on the values taken by the stock up to that date. Even though the path the stock price will take in the future is unknown, the derivative's price can be determined at the current time. For the special case of a European call or put option, Black and Scholes showed that "it is possible to create a hedged position, consisting of a long position in the stock and a short position in the option, whose value will not depend on the price of the stock". Their dynamic hedging strategy led to a partial differential equation which governs the price of the option. Its solution is given by the Black–Scholes formula.

## Notations

Let's define some notations.

### General and Market Parameters

$t$ is a time usually in years. $t=0$ is the current time.

$r$ is the annualized risk-free interest rate, continuously compounded.

### Asset Parameters

$S(t)$ is the price of the underlying asset at time $t$.

$\mu$  is the drift rate of $S$, annualized.

$\sigma$  is the standard deviation of the stock's returns. This is the square root of the quadratic variation of the stock's log price process, a measure of its volatility.

### Option Parameters

$V(S,t)$ is the price of the option as a function of the underlying asset $S$ at time $t$, in particular:
- $C(S,t)$ is the price of a European call option
- $P(S,t)$ is the price of a European put option

$T$ is the time to maturity of the option, in years.
$\tau = T-t$ is the time to maturity of the option, in years.
$K$ is the strike price of the option.

$N(x)$ denotes the standard normal cumulative distribution function:

${\displaystyle N(x)={\frac {1}{\sqrt {2\pi }}}\int _{-\infty }^{x}e^{-z^{2}/2}\,dz.}$

$N'(x)$ denotes the standard normal probability density function:

${\displaystyle N'(x)={\frac {dN(x)}{dx}}={\frac {1}{\sqrt {2\pi }}}e^{-x^{2}/2}.}$


## Differential Equation

BS equation describes the price of an option as a function of the underlying asset price $S$ and time $t$. The equation is a partial differential equation (PDE) of the second order in $S$ and the first order in $t$. We have

${\displaystyle {\frac {\partial V}{\partial t}}+rS{\frac {\partial V}{\partial S}}+{\frac {1}{2}}\sigma ^{2}S^{2}{\frac {\partial ^{2}V}{\partial S^{2}}}-rV=0}$

Let's rewrite the equation as:
$$
{\frac {\partial V}{\partial t}}+{\frac {1}{2}}\sigma ^{2}S^{2}{\frac {\partial ^{2}V}{\partial S^{2}}} = rV-rS{\frac {\partial V}{\partial S}}
$$

In terms of greeks, we have
$$
\Theta+{\frac {1}{2}}\sigma ^{2}S^{2}\Tau = rV-rS\Delta
$$

## Solution and the Black-Scholes Formula

To solve the equation, we need to specify the boundary conditions. The boundary conditions are the values of the option price at the boundaries of the domain of the option price. In the case of a European option, the boundary conditions are the payoff of the option at maturity. For a call option, the payoff is $S-K$ if $S>K$ and zero otherwise. For a put option, the payoff is $K-S$ if $S<K$ and zero otherwise. Therefore, the boundary conditions are

${\displaystyle C(S,T)=\max (S-K,0)}$

and

${\displaystyle P(S,T)=\max (K-S,0)}$

### Call Option

The value of a call option for a non-dividend-paying underlying stock in terms of the Black–Scholes parameters is:

$$
{\displaystyle {\begin{aligned}C(S_{t},t)&=N(d_{1})S_{t}-N(d_{2})Ke^{-r(T-t)}\\d_{1}&={\frac {1}{\sigma {\sqrt {T-t}}}}\left[\ln \left({\frac {S_{t}}{K}}\right)+\left(r+{\frac {\sigma ^{2}}{2}}\right)(T-t)\right]\\d_{2}&=d_{1}-\sigma {\sqrt {T-t}}\\\end{aligned}}}
$$

### Put Option

For a put option, the value is:

$$
{\displaystyle {\begin{aligned}P(S_{t},t)&=Ke^{-r(T-t)}-S_{t}+C(S_{t},t)\\&=N(-d_{2})Ke^{-r(T-t)}-N(-d_{1})S_{t}\end{aligned}}\,}
$$

The function $N(・)$ represents the cumulative distribution function for a normal distribution and may be thought of as ‘the probability that a random variable is less or equal to its input. See the terminology section above for more details.

>Very informally, the two terms in the sum given by the Black-Scholes formula may be thought of as ‘the current price of the stock weighted by the probability that you will exercise your option to buy the stock’ minus ‘the discounted price of exercising the option weighted by the probability that you will exercise the option’, or simply ‘what you are going to get’ minus ‘what you are going to pay’

### An Example Calculation

We see, from the formula that in order to calculate the price of a call option, we need to know the following parameters:
1. $S_{t}$, the current price of the underlying asset
2. $K$, the strike price of the option
3. $r$, the risk-free interest rate
4. $T-t$, the time to maturity of the option
5. $\sigma$, the volatility of the underlying asset

Here, we'll give an example of how to calculate the price of a European call option using the Black-Scholes formula. We'll use Tesla.

The current price of TSLA today (6 Jan 2023) is 110.34. We'll buy a call option trading at a strike price of 120. Suppose the option strike date of April 21 2022. We'll be using the US governments' 10Y bond rate as risk free rate. It is currently trading at 3.72% or 0.0372.

Only thing which remains is the volatility $\sigma$. We can estimate any stock’s volatility by observing its historical prices, or, even simpler, by calculating other option prices for the same stock at different maturity/expiration dates (T) and exercise/strike prices (X), if we know they have been set according to a Black-Scholes model. The resulting value, $\sigma$, is a number between 0 and 1, representing the market’s implied volatility for the stock. We'll talk about this in the next section. We'll calculate the volatility based on historical data. Using the log returns of the stock, we can calculate the volatility as follows:
$$
\sigma = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(\ln(S_{i})-\ln(S_{i-1}))^{2}}
$$
where $n$ is the number of observations, $S_{i}$ is the price of the stock at time $i$ and $S_{i-1}$ is the price of the stock at time $i-1$.

In [6]:
df = pd.read_csv("../../Data/Stocks/nasdaq200/TSLA.csv", parse_dates=["Date"], index_col="Date")
df = df[df.index>=datetime.datetime(2016,1,1)]
return_ = df["Close"].pct_change(1).dropna()
log_return = np.log(1+return_)
sigma = log_return.std()*np.sqrt(252)
print(sigma)

0.5854294125474527


In [7]:
today = datetime.datetime.today()
april_expiry_date = datetime.datetime(today.year, 4, 21)

num_days = (april_expiry_date - today).days
print(num_days)

103


In [8]:
S = 110.34
K = 120
r = 0.0342
tau = num_days/252

In [9]:
def d1(S, K, r, sigma, tau):
    return (np.log(S/K) + (r + sigma**2/2)*tau)/(sigma*np.sqrt(tau))

def d2(S, K, r, sigma, tau):
    return (np.log(S/K) + (r - sigma**2/2)*tau)/(sigma*np.sqrt(tau))

def call_price(sigma, S, K, r, tau):
    return S*norm.cdf(d1(S, K, r, sigma, tau)) - K*np.exp(-r*tau)*norm.cdf(d2(S, K, r, sigma, tau))

def put_price(sigma, S, K, r, tau):
    return K*np.exp(-r*tau)*norm.cdf(-d2(S, K, r, sigma, tau)) - S*norm.cdf(-d1(S, K, r, sigma, tau))

In [10]:
price = call_price(sigma, S, K, r, tau)
print(price)

13.267939948942484


Looking for the option chain at the internet, we find that the option is trading at 13.86. The difference is mainly because we have used the volatility calculated from historical data while the BS model uses implied volatilities, which may be different from historical volatilities.

## Python Implementation

Here, we'll do a simple implementation of the Black-Scholes formula in Python.

In [19]:
class BlackScholes:
    def __init__(self) -> None:
        pass

    def __d1(self, S, K, r, sigma, tau):
        multiplier = 1 / (sigma * np.sqrt(tau)+1e-10)
        term1 = np.log(S / K)
        term2 = (r + sigma**2 / 2) * tau
        return multiplier * (term1 + term2)

    def __d2(self, S, K, r, sigma, tau):
        d1 = self.__d1(S, K, r, sigma, tau)
        return d1 - sigma * np.sqrt(tau)

    def call(self, S, K, r, sigma, tau):
        """
        Calculates the price of a call option using the Black-Scholes model.

        Parameters
        ----------
        S : float
            The current price of the underlying asset.
        K : float
            The strike price of the option.
        r : float
            The risk-free interest rate.
        sigma : float
            The volatility of the underlying asset.
        tau : float
            The time to maturity of the option in years.

        Returns
        -------
        float
            The price of the call option.
        """
        d1 = self.__d1(S, K, r, sigma, tau)
        d2 = self.__d2(S, K, r, sigma, tau)
        term_1 = S * norm.cdf(d1)
        term_2 = K * np.exp(-r * tau) * norm.cdf(d2)
        return term_1 - term_2

    def put(self, S, K, r, sigma, tau):
        """
        Calculates the price of a put option using the Black-Scholes model.

        Parameters
        ----------
        S : float
            The current price of the underlying asset.
        K : float
            The strike price of the option.
        r : float
            The risk-free interest rate.
        sigma : float
            The volatility of the underlying asset.
        tau : float
            The time to maturity of the option in years.

        Returns
        -------
        float
            The price of the put option.
        """
        d1 = self.__d1(S, K, r, sigma, tau)
        d2 = self.__d2(S, K, r, sigma, tau)
        term_1 = K * np.exp(-r * tau) * norm.cdf(-d2)
        term_2 = S * norm.cdf(-d1)
        return term_1 - term_2

In [29]:
S = 110.34
K = 120
r = 0.0342
num_days = 1
tau = num_days/252
sigma = 0.5
bs = BlackScholes()
call_price = bs.call(S, K, r, sigma, tau)
put_price = bs.put(S, K, r, sigma, tau)
print("Call Price", call_price)
print("Put Price", put_price)
print(S+put_price)
print(S+call_price)

Call Price 0.004372032161472195
Put Price 9.648087422927802
119.9880874229278
110.34437203216147


In [27]:
S = 110.34
K = 100
r = 0.0342
num_days = 1
tau = num_days/252
sigma = 0.5
bs = BlackScholes()
call_price = bs.call(S, K, r, sigma, tau)
put_price = bs.put(S, K, r, sigma, tau)
print("Call Price", call_price)
print("Put Price", put_price)
print(S+put_price)

Call Price 10.354368191030673
Put Price 0.0007976833359694818
110.34079768333598


# Greeks

"The Greeks" measure the sensitivity of the value of a derivative product or a financial portfolio to changes in parameter values while holding the other parameters fixed. They are partial derivatives of the price with respect to the parameter values. For details, see the Options notebook.

The greeks can directly be calculated by taking derivatives of the Black–Scholes formula.

## Mathematical Form of Greeks

### Greeks for Call

**Delta $\mathbf{\left({\frac {\partial V}{\partial S}}\right)}$**

$$
N(d_1)
$$

**Gamma $\left(\mathbf{{\frac {\partial ^{2}V}{\partial S^{2}}}}\right)$**

$${\frac {N'(d_{1})}{S\sigma {\sqrt {T-t}}}}\$$

**Vega** $\left(\mathbf{{\displaystyle {\frac {\partial V}{\partial \sigma }}}}\right)$

$$SN'(d_{1}){\sqrt {T-t}}\,$$

**Theta** $\left(\mathbf{{\frac  {\partial V}{\partial t}}}\right)$

$${\displaystyle -{\frac {SN'(d_{1})\sigma }{2{\sqrt {T-t}}}}-rKe^{-r(T-t)}N(d_{2})\,}$$

**Rho** $\left(\mathbf{{\displaystyle {\frac {\partial V}{\partial r}}}}\right)$

$$K(T-t)e^{-r(T-t)}N(d_{2})\,$$

### Greeks for Put

**Delta $\mathbf{\left({\frac {\partial V}{\partial S}}\right)}$**

$$
-N(-d_{1})=N(d_{1})-1\,
$$

**Gamma $\left(\mathbf{{\frac {\partial ^{2}V}{\partial S^{2}}}}\right)$**

$${\frac {N'(d_{1})}{S\sigma {\sqrt {T-t}}}}\$$

**Vega** $\left(\mathbf{{\displaystyle {\frac {\partial V}{\partial \sigma }}}}\right)$

$$SN'(d_{1}){\sqrt {T-t}}\,$$

**Theta** $\left(\mathbf{{\frac  {\partial V}{\partial t}}}\right)$

$${\displaystyle -{\frac {SN'(d_{1})\sigma }{2{\sqrt {T-t}}}}+rKe^{-r(T-t)}N(-d_{2})\,}$$

**Rho** $\left(\mathbf{{\displaystyle {\frac {\partial V}{\partial r}}}}\right)$

$$-K(T-t)e^{-r(T-t)}N(-d_{2})\,$$

# Volatility

Volatility is a statistical measure of the dispersion of returns for a given security or market index. In most cases, the higher the volatility, the riskier the security. Volatility is often measured from either the standard deviation or variance between returns from that same security or market index.

## Log Returns

If we consider a conitnuous compounding model, the log return of a stock is given by:

$$
R_t = \ln \left( \frac{S_t}{S_{t-1}} \right)
$$

If $r_t$ is the usual return, then the log return is given by:
$$
R_t = \ln \left( 1 + r_t \right)
$$

This shows that $R$ is the continuously compounded return needed to produce the simple return $r$ during that period.

### Adding Log Returns

Simple returns can not be added. For example, suppose a stock is trading at 100. After a month, it goes to 120, which has a return of 0.20. Now, if the stock comes back to 100, the return is -0.167. The total return is 0.00, however, the simple returns add up to 0.033. Which is not zero!

In [7]:
r1 = 0.20
r2 = -0.167
print("Sum of Simple Returns: ", r1 + r2)

Sum of Simple Returns:  0.033


Now, let's use the log return:

In [8]:
R1 = np.log(1+r1)
R2 = np.log(1+r2)

print("Sum of Log Returns: ", R1 + R2)

Sum of Log Returns:  -0.0004000800213398159


This is very near to zero. In fact, if we set `r2` to be closer to 0.1666667, we get a value of zero.

In [9]:
r1 = 0.20
r2 = -0.1666667
R1 = np.log(1+r1)
R2 = np.log(1+r2)

print("Sum of Log Returns: ", R1 + R2)

Sum of Log Returns:  -4.0000000756101883e-08


> Note, however that unlike simple returns, you can not ad log return across securities of a portfolio in the same time period.

## Calculating Volatility

### Using Simple Returns

Simple returns can be used to calculate volatility. The formula is:
$$
\sigma = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n} (r_i - \bar{r})^2}
$$

where $n$ is the number of observations, $r_i$ is the return of the $i$th observation, and $\bar{r}$ is the mean of the returns.

In [30]:
df = pd.read_csv("../../Data/Stocks/nasdaq200/TSLA.csv", parse_dates=["Date"], index_col="Date")
df = df[df.index>=datetime.datetime(2016,1,1)]
return_ = df["Close"].pct_change(1).dropna()
sigma = return_.std()*np.sqrt(252)
print(sigma)

0.5860743591289022


So, the volatility of TSLA is 0.586 annually, which is 58.6% annually.

### Using the Log Return

We can use the standard deviation of the log returns to calculate the volatility of a stock. The standard deviation of the log returns is given by:
$$
\sigma = \sqrt{\frac{1}{T}\sum_{t=1}^T (R_t - \bar{R})^2}
$$

where $\bar{R}$ is the mean of the log returns.

In [31]:
df = pd.read_csv("../../Data/Stocks/nasdaq200/TSLA.csv", parse_dates=["Date"], index_col="Date")
df = df[df.index>=datetime.datetime(2016,1,1)]
return_ = df["Close"].pct_change(1).dropna()
log_return = np.log(1+return_)
sigma = log_return.std()*np.sqrt(252)
print(sigma)

0.5854294125474527


So, the volatility of TSLA is 0.585 annually, which is 58.5% annually.

### Using the Black-Scholes Model

If we use the BS model to solve for $\sigma$ by using the option prices, volatility can be calculated. This is done by using the `fsolve` function from `scipy.optimize`. The function takes in the parameters of the BS model and the option price, and returns the volatility.

In [11]:
from scipy.optimize import minimize
from functools import partial

In [16]:
def get_sigma(S, K, r, tau, current_price):
    call_price_final = partial(call_price, S=S, K=K, r=r, tau=tau)
    def error(sigma):
        return np.square(call_price_final(sigma) - current_price)
    return minimize(error, 0.2).x[0]

In [17]:
S = 110.34
K = 120
r = 0.0342
tau = num_days/252
current_price = 13.86
sigma = get_sigma(S, K, r, tau, current_price)
print("The implied volatility is: ", sigma)

The implied volatility is:  0.6064680543474251


We can see that the implied volatility is greater than the historical volatility.